In [1]:
#CNN 실습

In [2]:
#라이브러리 임포트

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [4]:
#데이터셋 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [8]:
#CIFAR-10 데이터셋 로드
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

100.0%


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [18]:
#간단한 CNN 모델 정의
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)  # 입력 채널 3, 출력 채널 32, 커널 크기 3x3
        self.pool = nn.MaxPool2d(2, 2)               # 풀링 크기 2x2
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) # 입력 채널 32, 출력 채널 64, 커널 크기 3x3
        self.fc1 = nn.Linear(64 * 8 * 8, 512)        # 완전 연결 층
        self.fc2 = nn.Linear(512, 10)                # 출력 층 (10개의 클래스)

    # forward 메서드 추가
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # 첫 번째 합성곱 -> ReLU -> 풀링
        x = self.pool(torch.relu(self.conv2(x)))  # 두 번째 합성곱 -> ReLU -> 풀링
        x = x.view(-1, 64 * 8 * 8)  # 데이터를 1차원으로 변환 (플래튼)
        x = torch.relu(self.fc1(x))  # 완전 연결 층 -> ReLU
        x = self.fc2(x)  # 최종 출력 층
        return x

In [19]:
#모델 학습

In [20]:
#모델 초기화
model = SimpleCNN()

In [21]:
#손실 함수와 최적화 알고리즘 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum = 0.9)

In [24]:
#모델 학습
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        #기울기 초기화
        optimizer.zero_grad()

        #순전파 + 여전파 + 최적화
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        #손실 출력
        running_loss += loss.item()
        if i % 100 == 99:
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

[Epoch 1, Batch 100] loss: 1.581
[Epoch 1, Batch 200] loss: 1.485
[Epoch 1, Batch 300] loss: 1.396
[Epoch 1, Batch 400] loss: 1.309
[Epoch 1, Batch 500] loss: 1.265
[Epoch 1, Batch 600] loss: 1.201
[Epoch 1, Batch 700] loss: 1.163
[Epoch 2, Batch 100] loss: 1.076
[Epoch 2, Batch 200] loss: 1.011
[Epoch 2, Batch 300] loss: 0.998
[Epoch 2, Batch 400] loss: 0.985
[Epoch 2, Batch 500] loss: 0.958
[Epoch 2, Batch 600] loss: 0.942
[Epoch 2, Batch 700] loss: 0.930
[Epoch 3, Batch 100] loss: 0.821
[Epoch 3, Batch 200] loss: 0.805
[Epoch 3, Batch 300] loss: 0.803
[Epoch 3, Batch 400] loss: 0.815
[Epoch 3, Batch 500] loss: 0.807
[Epoch 3, Batch 600] loss: 0.786
[Epoch 3, Batch 700] loss: 0.774
[Epoch 4, Batch 100] loss: 0.654
[Epoch 4, Batch 200] loss: 0.644
[Epoch 4, Batch 300] loss: 0.645
[Epoch 4, Batch 400] loss: 0.663
[Epoch 4, Batch 500] loss: 0.662
[Epoch 4, Batch 600] loss: 0.639
[Epoch 4, Batch 700] loss: 0.666
[Epoch 5, Batch 100] loss: 0.481
[Epoch 5, Batch 200] loss: 0.496
[Epoch 5, 

In [28]:
#모델 평가
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

Accuracy of the network on the 10000 test images: 73.63%
